# Using an HLS core in PYNQ

In this notebook we will finally interact with the HLS Core we wrote in [Building a Bitstream](3-Building-A-Bitstream.ipynb)

This notebook is divided into two parts: First, we load the block diagram built and bitstream compiled in the previous notebook. Second, we build a Python Package to install and re-use. 

**To skip this step**

Foobar

## Outputs from **[Building a Bitstream](3-Building-A-Bitstream.ipynb)**

The first two critical components of a PYNQ overlay are a `.tcl` script file and a bitfile. These files should have been created in **[Building a Bitstream](3-Building-A-Bitstream.ipynb)** and with the names `riscv.tcl` and `riscv.bit`.

Alternatively, you can skip this step by running the command below:

In [ ]:
!cp /home/xilinx/PYNQ-HLS/pynqhls/stream.tcl /home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/
!cp /home/xilinx/PYNQ-HLS/pynqhls/stream.bit /home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/

Otherwise, verify that the following files are in the `tutorial/pynqhls/stream` folder of your PYNQ-HLS repository on your **host computer**. 
       
   1. stream.tcl 
   2. stream.bit
   
Using [SAMBA](http://pynq.readthedocs.io/en/v2.0/getting_started.html#accessing-files-on-the-board), or SCP, copy these files from your host machine to the directory `/home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/` on your PYNQ board.


## Python Files

Before we verify that the `stream.tcl` and `stream.bit` files are working correctly, we need to create the Python Files that complete our Overlay. Two files are required: 

1. `__init__.py` to define the pynqhls Python package the overlay resides in
2. `stream.py` The python class that interacts with the FPGA bitstream

`__init__.py` is simple - so we will start there. 

Copy the following cell into a file named `__init__.py` in the `/home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/` directory on your PYNQ board. 

In [ ]:
from .stream import streamOverlay

This declares the streamOverlay python file to be part of the stream library. By residing in the `pynqhls` folder, it is part of the `pynqhls` package. 

Next, we create the `stream.py` file that defines the `streamOverlay` class as an interface for our FPGA Bitstream.

Copy and paste the following cell into a file named `stream.py` in the `/home/xilinx/PYNQ-HLS/tutorial/pynqhls/stream/` directory on your PYNQ board. 

In [ ]:
from pynq import Overlay, GPIO, Register
import os
import inspect
class streamOverlay(Overlay):
    """A simple Stream Overlay for PYNQ.                                                                                                                                                                                                                                                      
    This overlay is implemented with a single Streaming HLS Core fed by                                                                                                                                                                                                                       
    a DMA Engine                                                                                                                                                                                                                                                                              
                                                                                                                                                                                                                                                                                              
    """
    __RESET_VALUE = 0
    __NRESET_VALUE = 1
    
    """For convenince, we define a few offsets that are scraped                                                                                                                                                                               
    from the filt1d implementation header files."""
    __FILT1D_AP_CTRL_OFF = 0x00
    __FILT1D_GIE_OFF     = 0x04
    __FILT1D_IER_OFF     = 0x08
    __FILT1D_ISR_OFF     = 0x0C
    __FILT1D_COEFF_OFFS  = [0x10, 0x18,	0x20, 0x28,
                            0x30, 0x38,	0x40, 0x48,
                            0x50]
    __FILT1D_LENGTH_OFF  = 0x54
    def __init__(self, bitfile, **kwargs):
        """Initializes a new streamOverlay object.                                                                                                                                                                                                                                            
                                                                                                                                                                                                                                                                                              
        """
        # The following lines do some path searching to enable a 
        # PYNQ-Like API for Overlays. For example, without these 
        # lines you cannot call streamOverlay('stream.bit') because 
        # stream.bit is not on the bitstream search path. The 
        # following lines fix this for any non-PYNQ Overlay
        #
        # You can safely reuse, and ignore the following lines
        #
        # Get file path of the current class (i.e. /opt/python3.6/<...>/stream.py)                                                                                                                                                                                                            
        file_path = os.path.abspath(inspect.getfile(inspect.currentframe()))
        # Get directory path of the current class (i.e. /opt/python3.6/<...>/stream/)                                                                                                                                                                                                         
        dir_path = os.path.dirname(file_path)
        # Update the bitfile path to search in dir_path                                                                                                                                                                                                                                       
        bitfile = os.path.join(dir_path, bitfile)
        #
        # Upload the bitfile (and parse the colocated .tcl script)
        super().__init__(bitfile, **kwargs)
        # Manually define the GPIO pin that drives reset
        self.__resetPin = GPIO(GPIO.get_gpio_pin(0), "out")
        # Define a Register object at address 0x0 of the filt1d address space
        # We will use this to set bits and start the core (see start())
        # Do NOT write to __ap_ctrl unless __resetPin has been set to __NRESET_VALUE
        self.__ap_ctrl = Register(self.filt1d.mmio.base_addr, 32)                                                                                                                                                                                                                                                   

    def start(self):
        """ Toggle the startPin GPIO to drive AP_START and enable the HLS                                                                                                                                                                                                                     
        core"""
        self.__resetPin.write(__NRESET_VALUE)
        start = self.__ap_ctrl[0]
        
        # TODO: Start HLS Core by writing to AP_CTRL                                                                                                                                                                                                                                          
        pass

    def reset(self):
        """ Set the reset pin to self.__RESET_VALUE to place the core into                                                                                                                                                                                                                    
        reset"""
        self.__resetPin.write(self.__RESET_VALUE)

    def run(self, sbuf, dbuf):
        """ Launch computation on the HLS core                                                                                                                                                                                                                                                
                                                                                                                                                                                                                                                                                              
        Parameters                                                                                                                                                                                                                                                                            
        ----------                                                                                                                                                                                                                                                                            
        sbuf : ContiguousArray                                                                                                                                                                                                                                                                
            An xlnk allocated buffer to be transferred to the core                                                                                                                                                                                                                            
                                                                                                                                                                                                                                                                                              
        sbuf : ContiguousArray                                                                                                                                                                                                                                                                
            An xlnk allocated buffer to be transferred to the core                                                                                                                                                                                                                            
        """
        self.__resetPin.write(self.__NRESET_VALUE)
        self.start()
        # Load DMA Engine                                                                                                                                                                                                                                                                     
        # Wait for termination                                                                                                                                                                                                                                                                

    def load(self, coeffs):
        """ Load the filter coefficients into the HLS core """



In [1]:
import sys
sys.path.insert(0, '/home/xilinx/PYNQ-HLS/')

from pynqhls.stream import streamOverlay

overlay = streamOverlay('stream.bit')
#overlay.ip_dict

In [2]:
input = range(1, 100)
coeffs = [1] + [8] * 0
output = overlay.run(coeffs, input)

Not in reset!
Loaded coefficients!
Wrote Length!
Allocated Source CMA Array
Allocated Destination CMA Array
Started Transmit
Started Send
Pulsed Start!
Finished Send


KeyboardInterrupt: 

In [4]:
from pynq import Xlnk

Xlnk?

In [6]:
overlay.hlsDmaEngine.recvchannel.running

True

In [11]:
from pynq.xlnk import ContiguousArray

In [12]:
help(ContiguousArray)

Help on class ContiguousArray in module pynq.xlnk:

class ContiguousArray(numpy.ndarray)
 |  A subclass of numpy.ndarray which is allocated using
 |  physically contiguous memory for use with DMA engines and
 |  hardware accelerators. As physically contiguous memory is a
 |  limited resource it is strongly recommended to free the
 |  underlying buffer with `close` when the buffer is no longer
 |  needed. Alternatively a `with` statement can be used to
 |  automatically free the memory at the end of the scope.
 |  
 |  This class should not be constructed directly and instead
 |  created using `Xlnk.cma_array`.
 |  
 |  Attributes
 |  ----------
 |  pointer: cdata void*
 |      The virtual address pointer to the memory location
 |  physical_address: int
 |      The physical address to the array
 |  
 |  Method resolution order:
 |      ContiguousArray
 |      numpy.ndarray
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |  
 |  __enter__(self)
 |  
 |  __e

In [3]:
[1]  + [0]


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]